<!--
https://carpentries-incubator.github.io/geospatial-python/06-raster-intro.html
-->

# `rioxarray`

`rioxarray`:
-  is a Python extension for `xarray` to manipulate `xarray.DataArray`s as rasters. 
- means *raster input/output + xarray*. 

## Data
Raster filesfrom the US National Agriculture Imagery Program (NAIP)

- high-resolution aerial images from 2020 with four spectral bands: Red, Green, Blue and Near-infrared (NIR). 
- data is pre-processed into two rasters (RGB and NIR) 

## Import .tif

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import rioxarray as rioxr

import geopandas as gpd
from shapely.geometry import Polygon

*There are multiple ways of opening a '.tif' file using `xarray` or `rioxarray`*

Use `rioxarray.open_rasterio()` function to open the '.tif' file:

In [ ]:
# load NIR tif file
nir_fp = os.path.join(os.getcwd(),'data','naip','nir.tif')
nir = rioxr.open_rasterio(nir_fp)
nir

## `xr.DataArray` exploration

Let's verify the raster we loaded is an `xarray.DataArray`:

In [ ]:
type(nir)

Access some attribues:

In [ ]:
# print shape and data type
print('shape: ', nir.shape)
print('data type: ', nir.dtype, '\n')

The `.values` attribute gives a view at the values at the corners:

In [ ]:
# nir.values is a np.array: the underlying data
print(type(nir.values))
nir.values

We can also plot our data:

In [ ]:
# nir spectrum captured by sensor
nir.plot()

*Can you guess where this?*

## `rio` accessor

An **accessor** in Python let's us access a different set of properties of an object.

Use the `.rio` accessor for `xarray.DataArray`s to access its raster properties. 

For example:

In [ ]:
print('# bands: ', nir.rio.count)
print('height: ', nir.rio.height)
print('width: ', nir.rio.width, '\n')

print('spatial bounding box: ')
print(nir.rio.bounds(), '\n')

print('affine transformation: ')
print(nir.rio.transform(), '\n')

print('CRS: ', nir.rio.crs)

*We have used accessors before, for example the `.str` and `.dt` accessors in `pandas`.*

## Multi-band raster

Import the RGB raster:

In [ ]:
# open RGB raster
rgb_fp = os.path.join(os.getcwd(),'data','naip','rgb.tif')
rgb = rioxr.open_rasterio(rgb_fp)
rgb

*Raster has three bands, instead of one. This makes sense because we know these bands correspond tothe Red, Green and Blue bands of the image.*

Check number of bands and shape:

In [ ]:
print('rgb shape: ', rgb.shape)
print('rgb # bands: ', rgb.rio.count)

In [ ]:
# check geospatial data
print('shape: ', rgb.shape)
print('data type: ', rgb.dtype)
print('# bands: ', rgb.rio.count)
print('CRS: ', rgb.rio.crs)

# check if the CRSs of the rasters match
print( rgb.rio.crs == nir.rio.crs)

Plot raster

Since it has three bands, we can plot it as an image using the `.plot.imshow()` method, which will interpret the three bands of the object as RGB:

In [ ]:
# plot three bands as RGB image
rgb.plot.imshow(figsize=(8,8))

: 

## Box for clipping

Our area of interest (aoi) is a smaller region with a few blocks around the NCEAS building. 


*Go to https://geojson.io/ to get points*

In [ ]:
# vertices of our aoi box
points = [[-119.70608227128903, 34.426300194372274],
          [-119.70608227128903, 34.42041139020533],
          [-119.6967885126002, 34.42041139020533],
          [-119.6967885126002, 34.426300194372274],
          [-119.70608227128903, 34.426300194372274]]

We can then create a new `geopandas.GeoDataFrame`:

In [ ]:
# create geodataframe with aoi 
aoi = gpd.GeoDataFrame(geometry=[Polygon(points)],
                           crs='epsg:4326')
aoi

Let's break this down a bit:

- first, we use the `shapely`'s `Polygon()` function to create a polygon from our `points` list. 
- in `[Polygon(points)]` we put this polygon inside a list so we can form the geometry column of our new `gpd.GeoDataFrame`
- we know all the geoJSON files have CRS equal to EPSG:4326/WGS 84, so we set the the CRS of our new `gpd.GeoDataFrame` to this.

## Clip raster

Remember: **if two geospatial sets will interact they need to be in the same CRS**.  
In our case, the aoi `gpd.GeoDataFrame` does not have the same CRS as the rasters:

In [ ]:
print('aoi CRS: ', aoi.crs)
print('nir CRS: ', nir.rio.crs)
print('rgb CRS: ', rgb.rio.crs)

So let's reproject:

In [ ]:
aoi = aoi.to_crs(rgb.rio.crs)
print('matched crs?',  aoi.crs == rgb.rio.crs)
aoi.crs

And plot them together:

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches((8,8))
rgb.plot.imshow(ax=ax)
aoi.plot(ax=ax, alpha=0.6)

To clip the raster using the aoi polygon we use the `.rio.clip_box()` method:

In [ ]:
# clip rasters to aoi
rgb_small = rgb.rio.clip_box(*aoi.total_bounds)
nir_small = nir.rio.clip_box(*aoi.total_bounds)

Notice a few things:
- we had to use the `.rio` accessor to access the `clip_box()` method 
- similarly to the `shapely.box()` function [we've used previously](https://carmengg.github.io/eds-220-book/lectures/lesson-13-standin.html#clipping-with-bounding-box), [`.rio.clip_box()` usual parameters](https://corteva.github.io/rioxarray/stable/rioxarray.html#rioxarray.raster_dataset.RasterDataset.clip_box) are minx, miny, maxx, maxy. We are using the `*` asterisk as an unpacking operator to get these from the list `aoi.total_bounds`.

Let's check our clipped data:

In [ ]:
print('original shape: ', rgb.shape)
print('reduced shape: ', rgb_small.shape)
rgb_small.plot.imshow()

In [ ]:
print('original shape: ', nir.shape)
print('reduced shape: ', nir_small.shape)
nir_small.plot()

## Compute NDVI

We often want to combine values of and perform calculations on rasters to create a new output raster. 
In our case, we are interested in computing the Normalized Difference Vegetation Index (NDVI) over our area of interest. 
The NDVI is an index commonly used to check if an area has live green vegetation or not.

According to the [Earth Observing System](https://eos.com/blog/ndvi-faq-all-you-need-to-know-about-ndvi/)
> The results of the NDVI calculation range from -1 to 1. Negative values correspond to areas with water surfaces, manmade structures, rocks, clouds, snow; bare soil usually falls within 0.1-0.2 range; and plants will always have positive values between 0.2 and 1. Healthy, dense vegetation canopy should be above 0.5, and sparse vegetation will most likely fall within 0.2 to 0.5. 

The NDVI is calculated using the NIR and red bands. 
The formula is

$NDVI = \frac{NIR - Red}{NIR + Red}.$

First, we need to select the red band:

In [ ]:
red = rgb_small.sel(band=1)
red

To be able to perform the calculation succefully, we will need to udpate the data type of our rasters:

In [ ]:
red16 = red.astype('int16')
nir16 = nir_small.astype('int16')
print('RED: original dtype:', rgb_small.dtype, '.... converted dtype:', red16.dtype)
print('NIR: original dtype:', nir.dtype, '.... converted dtype:', nir16.dtype)

We can perform raster calculations using the same arithmetic we use for `np.array`s (because, underneath it all, they are). 
So our NDVI calculation is as follows:

In [ ]:
# calculate and plot NDVI
ndvi = (nir16 - red16)/(nir16+red16)
ndvi.plot()

Remember that plants will always have positive NDVI values between 0.2 and 1. 
Can you spot the Courthouse?

:::{.calllout-caution collapse="true"}
# Why change the data type?
The `uint8` (8-bit unsigned integer) is a very small data type that only holds integers from 0 up to 255. 
In particular, calculations don't  return what we'd expect ([they're done module 256](https://en.wikipedia.org/wiki/Modular_arithmetic)):

In [ ]:
np.uint8(150) + np.uint8(150)

Notice that in the NDVI formula we have to add NIR + Red. 
If both NIR and Red are very close to 255, when we add them this overflows the `unit8` data type and we get garbage out of the calculation:

In [ ]:
x = (nir - red)/(nir + red)
x.plot()

This is why we need to manually convert both rasters into `int16`, which will be big enough to hold all the numbers that appear in the calculations.

Notice too, that when we performed the ndvi calculation we did not get any warning although we were overflowing the computation at every cell of our array. 
This is can be an example of *failing silently*, where we don't get any warnings about the errors in our computation. 
That's why it's so important to double-check our results!
:::